In [ ]:
import nltk
nltk.download("punkt_tab")
nltk.download("averaged_perceptron_tagger_eng")

import warnings
warnings.simplefilter("ignore")

import os
import sys
import tiktoken
from glob import glob
from tqdm.auto import tqdm
from time import sleep
from dotenv import load_dotenv

import polars as pl

from langchain.chains import RetrievalQA
from langchain_openai import AzureChatOpenAI, AzureOpenAIEmbeddings
from langchain.prompts import PromptTemplate
from langchain.output_parsers import StructuredOutputParser, ResponseSchema

sys.path.append("..")
from src.dataset.postprocess import process_markdown_file  # noqa: E402
from src.tools.create_docs import process_files_in_batches  # noqa: E402
from src.model.retriever import create_retriever  # noqa: E402

load_dotenv()

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/kwatanabe/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /home/kwatanabe/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


True

## Markdownファイルについて後処理を実施
- 必要のない文章を含む行を指定して削除

In [ ]:
val_md_paths_bef = sorted(glob("../data/documents/gpt_4omini_markdowns/*.md"))
for val_md_path in val_md_paths_bef:
    process_markdown_file(
        input_file=val_md_path,
        line_target_words=["統合報告書", "統合レポート"],
        header_keywords=["INDEX", "目次"],
        output_dir="../data/documents/gpt_4omini_markdowns/postprocess"
    )

test_md_paths_bef = sorted(glob("../data/test/documents/markdowns/*.md"))
for test_md_path in test_md_paths_bef:
    process_markdown_file(
        input_file=test_md_path,
        line_target_words=["統合報告書", "統合レポート"],
        header_keywords=["INDEX", "目次"],
        output_dir="../data/test/documents/markdowns/postprocess"
    )

In [ ]:
val_query = pl.read_csv("../signate_data/validation/ans_txt.csv")
val_md_paths = sorted(glob("../data/documents/gpt_4omini_markdowns/postprocess/*.md"))
test_query = pl.read_csv("../signate_data/query.csv")
test_md_paths = sorted(glob("../data/test/documents/markdowns/postprocess/*.md"))

## Vector DBを作成
- バッチ処理による文書のVector DB格納作業を実施

In [ ]:
embeddings = AzureOpenAIEmbeddings(
    model=os.getenv("EMBEDDING")
)

target_words = ["統合報告書", "統合レポート"]
header_words = ["INDEX", "目次"]
chunk_size = 500
chunk_overlap = 0

val_vector_store = process_files_in_batches(
    embeddings=embeddings,
    md_paths=val_md_paths,
    target_words=target_words,
    header_words=header_words,
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap
    )

test_vector_store = process_files_in_batches(
    embeddings=embeddings,
    md_paths=test_md_paths,
    target_words=target_words,
    header_words=header_words,
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap
    )

## Retrieverの作成

In [156]:
retriever_config = {
    "topk": 30,
    "hybrid": True,
    "hybrid_topk": 30,
    "hybrid_weights": [0.5, 0.5],
    "rerank": False,
    "rerank_topk": 10
}

val_retriever = create_retriever(
    vector_store=val_vector_store,
    topk=retriever_config["topk"],
    hybrid=retriever_config["hybrid"],
    hybrid_topk=retriever_config["hybrid_topk"],
    hybrid_weights=retriever_config["hybrid_weights"],
    rerank=retriever_config["rerank"],
    rerank_topk=retriever_config["rerank_topk"]
)

test_retriever = create_retriever(
    vector_store=test_vector_store,
    topk=retriever_config["topk"],
    hybrid=retriever_config["hybrid"],
    hybrid_topk=retriever_config["hybrid_topk"],
    hybrid_weights=retriever_config["hybrid_weights"],
    rerank=retriever_config["rerank"],
    rerank_topk=retriever_config["rerank_topk"]
)

## Promptの作成

In [ ]:
response_schemas = [
    ResponseSchema(
        name="answer",
        description="質問に対しその回答となる要素のみ（前後の文章は一切不要）を端的に出力する"
        )
]
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

qa_prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=("""
        あなたは、簡潔で正確な回答を生成するAIアシスタントです。以下のルールを**絶対に遵守**してください。


        ### **⚠ 絶対に守るべきルール**
        - 質問内に特定の指示（例：「四捨五入して答えよ」「株式会社をつけて」など）がある場合、**その指示に従って回答する**
        - 回答は必ず **54トークン以内** で出力する
        - 必要があれば読点を用いて横並びに出力する
        - **回答に数値を含む場合は、単位を明記** すること（例：「36拠点」、「3.0％」）
        - **質問に対して直接的な回答のみを出力する**（「以下が回答です：」のような前置きは不要）
        - 参照する情報に不備があり、回答に確証が得られない場合は必ず「不明」と出力する

        ### ** 入力例 **
        質問: A社の収益率は2018年度と2019年度ではどちらの数値が高いか
        回答: 2018年度

        質問: C社の支店数は何支店ですか？
        回答: 39支店

        質問: D社の2019年度の利益率は2018年度に比べて何%向上したか、少数第二位を四捨五入して答えよ
        回答: 21.2%

        質問: E社の2024年度の売上高は何億円になると予測できますか？
        回答: 83億円

        出力フォーマット: {format_instructions}
        情報: {context}
        質問: {question}
        回答:
    """
    ),
    partial_variables={"format_instructions": output_parser.get_format_instructions()}
)

def setup_qa_chain(client, retriever, prompt):
    qa_chain = RetrievalQA.from_chain_type(
        llm=client,
        chain_type="stuff",
        retriever=retriever,
        chain_type_kwargs={"prompt": prompt},
        return_source_documents=True
    )
    return qa_chain

In [ ]:
client = AzureChatOpenAI(
    openai_api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    deployment_name=os.getenv("MODEL"),
    api_version=os.getenv("AZURE_OPENAI_API_VERSION"),
    temperature=0,
    top_p=1,
    max_tokens=54,
)

val_qa_chain = setup_qa_chain(client, val_retriever, qa_prompt)
test_qa_chain = setup_qa_chain(client, test_retriever, qa_prompt)

In [ ]:
def create_answers(query, qa_chain, max_retries=3):
    encoding = tiktoken.get_encoding("cl100k_base")

    indices = [i for i in range(len(query))]
    answers = []
    source_documents = []
    for i, p in tqdm(enumerate(query["problem"]), total=len(query)):
        retries = 0
        while retries < max_retries:
            try:
                qa = qa_chain.invoke({"query": p})
                break
            except Exception as e:
                print(f"Attempt {retries+1} failed with error: {e}")
                retries += 1
                sleep(2 ** retries)
        else:
            print(f"Failed to process problem {i} after {max_retries} attempts.")
        try:
            answer = output_parser.parse(qa["result"])["answer"]
            token = encoding.encode(answer)
            if len(token) > 54:
                answer = "不明"
        except Exception as e:
            print(f"Error parsing response for query {i}: {e}")
            answer = "Error"
        print(answer)
        answers.append(answer)
        source_documents.append(qa["source_documents"])

    df = pl.DataFrame(
        data={
            "index": indices,
            "answer": answers
        },
        schema={
            "index": pl.UInt32,
            "answer": pl.String
        }
    )
    return df, source_documents

def save_csv(df, output_path):
    csv_data = df.write_csv().split("\n", 1)[-1]
    with open(output_path, "w") as f:
        f.write(csv_data)

## 検証データにおける回答生成

In [ ]:
val_df, val_sd = create_answers(query=val_query, qa_chain=val_qa_chain)
save_csv(df=val_df, output_path="../signate_data/evaluation/submit/predictions.csv")

## 検証データのスコア計算

In [ ]:
%run -i ../signate_data/evaluation/crag.py \
    --result-dir ../signate_data/evaluation/submit \
    --ans-dir ../signate_data/evaluation/data \
    --eval-result-dir ../signate_data/evaluation/result

## テストデータにおける回答生成

In [ ]:
test_df, test_sd = create_answers(query=test_query, qa_chain=test_qa_chain)
save_csv(df=test_df, output_path="../data/test/submit/predictions.csv")